In [ ]:
input_file1 = "QM9_129440_MLtraining"
input_file2 = "QM9_49762_MLtraining"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pickle
import os
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Đọc dữ liệu
df2 = pd.read_csv(f'{input_file2}.csv').replace([np.inf, -np.inf], np.nan).dropna()
df1 = pd.read_csv(f'{input_file1}.csv')

# Khởi tạo dictionary để lưu features
final_features = {}

for target in ['bps_pred', 'mps_pred', 'fps_pred']:
    X = df2.drop(columns=['bps_pred', 'mps_pred', 'fps_pred', 'canonical_smiles'])
    y = df2[target]
    
    # 1. Feature Importance từ Random Forest
    rf = RandomForestRegressor(n_estimators=100, random_state=7)
    rf.fit(X, y)
    rf_imp = pd.DataFrame({'feature': X.columns, 'RF_score': rf.feature_importances_})
    # 1.5. Thêm XGBoost Feature Importance
    xgb = XGBRegressor(random_state=7)
    xgb.fit(X, y)
    xgb_imp = pd.DataFrame({'feature': X.columns, 'XGB_score': xgb.feature_importances_})
    # 2. SelectKBest với Mutual Info
    selector = SelectKBest(mutual_info_regression, k='all')
    selector.fit(X, y)
    skb_imp = pd.DataFrame({'feature': X.columns, 'SKB_score': selector.scores_})
    
    # 3. PCA Loadings
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=0.9999)
    pca.fit(X_scaled)
    pca_loadings = pd.DataFrame(pca.components_.T, index=X.columns).abs().sum(axis=1)
    pca_imp = pd.DataFrame({'feature': pca_loadings.index, 'PCA_score': pca_loadings.values})
    
    # Kết hợp tất cả các phương pháp
    combined = rf_imp.merge(skb_imp, on='feature').merge(pca_imp, on='feature').merge(xgb_imp, on='feature', how='left')
    
    # Tính rank cho từng phương pháp
    for method in ['RF_score', 'SKB_score', 'PCA_score', 'XGB_score']:
        combined[f'{method}_rank'] = combined[method].rank(ascending=False, method='dense')
    
    # Tính rank trung bình
    combined['Avg_Rank'] = combined[['RF_score_rank', 'SKB_score_rank', 'PCA_score_rank', 'XGB_score_rank']].mean(axis=1)
    
    # Chọn top 20 features
    top_20 = combined.sort_values('Avg_Rank').head(20)['feature'].tolist()
    final_features[target] = top_20
    print(f"Top 20 features for {target}:")
    print(top_20)
    

# Lưu features vào list (ví dụ cho bps_pred)
BPS_features = final_features['bps_pred']
MPS_features = final_features['mps_pred']
FPS_features = final_features['fps_pred']

In [ ]:
def save_model(model_name, model_type, method):
    pkl_filename = f'{model_type}_{type(model_name).__name__}_{method}.pkl'
    with open(pkl_filename, 'wb') as file:  
        pickle.dump(model_name, file)
    with open(pkl_filename, 'rb') as file:  
        saved_model = pickle.load(file)
    print(saved_model)

# Tính toán MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Giả sử df2 là dataframe chứa dữ liệu của bạn
features = [col for col in df2.columns if col not in 
            ['canonical_smiles', 'mps_pred', 'bps_pred', 'fps_pred']]

targets = ['bps_pred', 'mps_pred', 'fps_pred']
target_features = {
    'bps_pred': BPS_features,
    'mps_pred': MPS_features,
    'fps_pred': FPS_features
}
methods = ['BPS', 'MPS', 'FPS']
cv = 5

# Hàm để lấy feature importance
def get_feature_importance(model, feature_names):
    if hasattr(model.named_steps['model'], 'feature_importances_'):
        importances = model.named_steps['model'].feature_importances_
    elif hasattr(model.named_steps['model'], 'coef_'):
        importances = model.named_steps['model'].coef_
    else:
        return None
    
    feature_importance = pd.DataFrame({'feature': feature_names, 
                                     'importance': importances})
    return feature_importance.sort_values('importance', ascending=False)

# Xây dựng mô hình cho từng target
for i, target in enumerate(targets):
    print(f"\n=== Building models for {target} ===")
    print("="*50)
    
    selected_features = target_features[target]
    X = df2[selected_features]
    # Handle infinity values by replacing with mean
    inf_mask = X.isin([np.inf, -np.inf])
    if inf_mask.any().any():
        X = X.replace([np.inf, -np.inf], np.nan)
        X = X.fillna(X.mean())
    y = df2[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
    
    # 1. Pipeline và param grid cho Random Forest
    rf_pipe = Pipeline([
        ("scaler", MinMaxScaler()), 
        ('model', RandomForestRegressor(criterion='absolute_error', random_state=7))
    ])
    
    rf_param_grid = {
        "model__n_estimators": [100],
        "model__max_depth": [20],
        "model__min_samples_leaf": [1],
    }
    
    # 2. Pipeline và param grid cho XGBoost
    xgb_pipe = Pipeline([
        # ("scaler", MinMaxScaler()), 
        ('model', XGBRegressor(objective='reg:absoluteerror', random_state=7))
    ])
    
    xgb_param_grid = {
        "model__n_estimators": [2000, 1000, 750],
        "model__max_depth": [8, 10, 12],
        "model__learning_rate": [0.09, 0.06],
    }
    
    models = [
        # ('Random Forest', rf_pipe, rf_param_grid),
        ('XGBoost', xgb_pipe, xgb_param_grid)
    ]
    
    best_models = {}
    
    for name, pipe, param_grid in models:
        print(f'\n----- Optimizing {name} for {methods[i]} -----')
        search = GridSearchCV(pipe, param_grid, n_jobs=-1, 
                            scoring='neg_mean_absolute_error')
        search.fit(X_train, y_train)
        
        best_model = search.best_estimator_
        y_pred = best_model.predict(X_test)
        
        # Tính các metrics
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        
        print(f"\nBest parameters for {name}:")
        for param, value in search.best_params_.items():
            print(f"{param}: {value}")
        
        print("\nEvaluation Metrics on Test Set:")
        print(f"R2 Score: {r2:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"MAPE: {mape:.4f}%")
        
        best_models[name] = {
            'model': best_model,
            'metrics': {
                'R2': r2,
                'RMSE': rmse,
                'MAE': mae,
                'MAPE': mape
            }
        }
    
    # So sánh kết quả tốt nhất giữa 2 mô hình
    print('\n----- Model Comparison -----')
    print(f"{'Model':<15} {'R2':>8} {'RMSE':>8} {'MAE':>8} {'MAPE':>8}")
    for name, result in best_models.items():
        metrics = result['metrics']
        print(f"{name:<15} {metrics['R2']:8.4f} {metrics['RMSE']:8.4f} " 
              f"{metrics['MAE']:8.4f} {metrics['MAPE']:8.4f}%")
    
    # Xác định best model dựa trên MAE
    best_model_name = min(best_models.items(), 
                         key=lambda x: x[1]['metrics']['MAE'])[0]
    print(f"\nBest model for {target} is {best_model_name} "
          f"(MAE = {best_models[best_model_name]['metrics']['MAE']:.4f})")
    
    # Lưu best model
    model_name = f"{methods[i]}_{best_model_name.replace(' ', '_')}"
    save_model(best_models[best_model_name]['model'], 'regression', methods[i])